In [6]:
import sys
sys.path.append('On_Multiple_Semantics_for_Declarative_Database_Repairs/')
import On_Multiple_Semantics_for_Declarative_Database_Repairs

from database_generator.dba import DatabaseEngine
from Semantics.end_sem import EndSemantics
from Semantics.stage_sem import StageSemantics
from Semantics.step_sem import StepSemantics
from Semantics.independent_sem import IndependentSemantics

# specify the schema
mas_schema = {"synthetic_data_version_1": ('RAC1P', 'SEX', 'REGION', 'PINCP', 'ST', 'COW', 'CIT', 'NATIVITY', 'MSP', 'SCHL', 'DIS','MIL')
              }


def read_rules(rule_file):
    """read programs from txt file"""
    all_programs = []
    with open(rule_file) as f:
        rules = []
        for line in f:
            if line.strip():
                tbl, r = line.split("|")
                rules.append((tbl, r[:-2]))
            else:
                all_programs.append([r for r in rules])
                rules = []
        all_programs.append(rules)
    return all_programs

# load delta programs
programs = read_rules("On_Multiple_Semantics_for_Declarative_Database_Repairs/../delta_program.txt")

# start the database
db = DatabaseEngine("cr")

tbl_names = ["synthetic_data_version_1"]
subfolder = "/datasets/muse/"

def database_reset(db):
    """reset the database"""
    res = db.execute_query("select current_database();")
    db_name = res[0][0]
    db.delete_tables(tbl_names)
    # db.close_connection()
    db = DatabaseEngine(db_name)
    db.load_database_tables(tbl_names)

# choose a delta program from the programs file
rules = programs[0]
print("delta program:", rules)

# # reset the database between runs
# database_reset(db)

# end_sem = EndSemantics(db, rules, tbl_names)
# end_semantics_result = end_sem.find_mss()
# print("result for end semantics:", end_semantics_result)

# # reset the database between runs
# database_reset(db)

# stage_sem = StageSemantics(db, rules, tbl_names)
# stage_semantics_result = stage_sem.find_mss()
# print("result for stage semantics:", stage_semantics_result)

# reset the database between runs
database_reset(db)

step_sem = StepSemantics(db, rules, tbl_names)
step_semantics_result = step_sem.find_mss(mas_schema)
print("result for step semantics:", step_semantics_result)

# reset the database between runs
# database_reset(db)

# ind_sem = IndependentSemantics(db, rules, tbl_names)
# ind_semantics_result = ind_sem.find_mss(mas_schema)
# print("result for independent semantics:", ind_semantics_result)


delta program: [('synthetic_data_version_1', 'SELECT synthetic_data_version_11.* FROM synthetic_data_version_1 AS synthetic_data_version_11, synthetic_data_version_1 AS synthetic_data_version_12 WHERE synthetic_data_version_11.ST = synthetic_data_version_12.ST AND synthetic_data_version_11.REGION <> synthetic_data_version_12.REGION')]
result for step semantics: {('synthetic_data_version_1', (6, 2, 4, 'None', 6, '6.0', 1, 1, '1.0', '18.0', 2, '4.0')), ('synthetic_data_version_1', (6, 2, 4, '-8500.0', 48, 'None', 1, 1, '4.0', '19.0', 2, '4.0')), ('synthetic_data_version_1', (1, 1, 1, '143922.0099869498', 51, '1.0', 1, 1, '1.0', '22.0', 2, '4.0')), ('synthetic_data_version_1', (1, 1, 4, '8020.964113451177', 45, '1.0', 1, 1, '6.0', '19.0', 2, '4.0')), ('synthetic_data_version_1', (1, 1, 2, 'None', 1, '1.0', 1, 1, 'None', '19.0', 2, '4.0')), ('synthetic_data_version_1', (1, 1, 1, '36409.89302439781', 34, 'None', 1, 1, '6.0', '21.0', 2, '4.0')), ('synthetic_data_version_1', (1, 2, 2, 'None',

In [7]:
len(step_semantics_result)

618

In [8]:
import pandas as pd
import psycopg2

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(
    database="cr",
    user="postgres",
    password="postgres"
)

# Execute an SQL query and create a dataframe hospital_repaired
muse_data_step = pd.read_sql("SELECT * FROM synthetic_data_version_1", conn).iloc[1:]
# muse_data_step.drop(muse_data_step.index[0])
muse_data_step.columns = muse_data_step.columns.str.upper()

conn.close()
display(muse_data_step)

/Users/kushagraghosh/opt/anaconda3/lib/python3.8/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,RAC1P,SEX,REGION,PINCP,ST,COW,CIT,NATIVITY,MSP,SCHL,DIS,MIL
1,6,2,3,-2615.090605818703,48,None,1,1,5.0,22.0,2,4.0
2,1,2,9,None,56,1.0,1,1,None,22.0,2,4.0
3,2,2,3,28787.55376344581,53,4.0,1,1,4.0,18.0,2,4.0
4,9,2,3,None,12,1.0,4,1,1.0,16.0,2,None
5,9,2,3,-8500.0,39,1.0,1,1,4.0,None,2,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
996,1,2,1,-8500.0,50,None,1,1,6.0,1.0,2,1.0
997,1,1,4,-6025.412259414399,39,1.0,1,1,1.0,21.0,2,4.0
998,8,2,3,None,37,1.0,4,1,6.0,16.0,2,4.0
999,1,1,4,1998.8384159913767,55,1.0,1,1,6.0,14.0,2,4.0


In [9]:
def make_col_int(df, *args):
    for col in args:
        df[col] = df[col].astype(float)
        df[col] = df[col].astype(int)
def make_col_string(df, *args):
    for col in args:
        df[col] = df[col].astype(str)
        
make_col_int(muse_data_step, 'RAC1P', 'SEX', 'REGION', 'ST', 'CIT', 'NATIVITY', 'DIS')
make_col_string(muse_data_step, 'PINCP', 'COW', 'MSP', 'SCHL', 'MIL')
print(muse_data_step.dtypes)

display(muse_data_step)

RAC1P        int64
SEX          int64
REGION       int64
PINCP       object
ST           int64
COW         object
CIT          int64
NATIVITY     int64
MSP         object
SCHL        object
DIS          int64
MIL         object
dtype: object


,RAC1P,SEX,REGION,PINCP,ST,COW,CIT,NATIVITY,MSP,SCHL,DIS,MIL
1,6,2,3,-2615.090605818703,48,None,1,1,5.0,22.0,2,4.0
2,1,2,9,None,56,1.0,1,1,None,22.0,2,4.0
3,2,2,3,28787.55376344581,53,4.0,1,1,4.0,18.0,2,4.0
4,9,2,3,None,12,1.0,4,1,1.0,16.0,2,None
5,9,2,3,-8500.0,39,1.0,1,1,4.0,None,2,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
996,1,2,1,-8500.0,50,None,1,1,6.0,1.0,2,1.0
997,1,1,4,-6025.412259414399,39,1.0,1,1,1.0,21.0,2,4.0
998,8,2,3,None,37,1.0,4,1,6.0,16.0,2,4.0
999,1,1,4,1998.8384159913767,55,1.0,1,1,6.0,14.0,2,4.0


In [10]:
tuples = [tup[1] for tup in step_semantics_result]
for t in tuples:
    row_index = muse_data_step.index[(muse_data_step == t).all(axis=1)]
    print(row_index)
    muse_data_step = muse_data_step.drop(row_index)

Int64Index([41], dtype='int64')
Int64Index([425], dtype='int64')
Int64Index([124], dtype='int64')
Int64Index([588], dtype='int64')
Int64Index([647], dtype='int64')
Int64Index([354], dtype='int64')
Int64Index([622], dtype='int64')
Int64Index([28], dtype='int64')
Int64Index([345], dtype='int64')
Int64Index([200], dtype='int64')
Int64Index([732], dtype='int64')
Int64Index([687], dtype='int64')
Int64Index([2], dtype='int64')
Int64Index([182], dtype='int64')
Int64Index([286], dtype='int64')
Int64Index([196], dtype='int64')
Int64Index([235], dtype='int64')
Int64Index([698], dtype='int64')
Int64Index([645], dtype='int64')
Int64Index([279], dtype='int64')
Int64Index([329], dtype='int64')
Int64Index([48], dtype='int64')
Int64Index([485], dtype='int64')
Int64Index([852], dtype='int64')
Int64Index([544], dtype='int64')
Int64Index([473], dtype='int64')
Int64Index([854], dtype='int64')
Int64Index([810], dtype='int64')
Int64Index([187], dtype='int64')
Int64Index([95], dtype='int64')
Int64Index([288]

Int64Index([432], dtype='int64')
Int64Index([214], dtype='int64')
Int64Index([482], dtype='int64')
Int64Index([209], dtype='int64')
Int64Index([307], dtype='int64')
Int64Index([219], dtype='int64')
Int64Index([816], dtype='int64')
Int64Index([813], dtype='int64')
Int64Index([408], dtype='int64')
Int64Index([538], dtype='int64')
Int64Index([696], dtype='int64')
Int64Index([856], dtype='int64')
Int64Index([86], dtype='int64')
Int64Index([707], dtype='int64')
Int64Index([151], dtype='int64')
Int64Index([90], dtype='int64')
Int64Index([610], dtype='int64')
Int64Index([168], dtype='int64')
Int64Index([114], dtype='int64')
Int64Index([646], dtype='int64')
Int64Index([542], dtype='int64')
Int64Index([675], dtype='int64')
Int64Index([857], dtype='int64')
Int64Index([953], dtype='int64')
Int64Index([476], dtype='int64')
Int64Index([96], dtype='int64')
Int64Index([657], dtype='int64')
Int64Index([446], dtype='int64')
Int64Index([618], dtype='int64')
Int64Index([908], dtype='int64')
Int64Index([8

Int64Index([98], dtype='int64')
Int64Index([592], dtype='int64')
Int64Index([545], dtype='int64')
Int64Index([806], dtype='int64')
Int64Index([938], dtype='int64')
Int64Index([528], dtype='int64')
Int64Index([568], dtype='int64')
Int64Index([423], dtype='int64')
Int64Index([965], dtype='int64')
Int64Index([232], dtype='int64')
Int64Index([259], dtype='int64')
Int64Index([574], dtype='int64')
Int64Index([581], dtype='int64')
Int64Index([392], dtype='int64')
Int64Index([824], dtype='int64')
Int64Index([306], dtype='int64')
Int64Index([976], dtype='int64')
Int64Index([683], dtype='int64')
Int64Index([985], dtype='int64')
Int64Index([13], dtype='int64')
Int64Index([506], dtype='int64')
Int64Index([899], dtype='int64')
Int64Index([582], dtype='int64')
Int64Index([273], dtype='int64')
Int64Index([704], dtype='int64')
Int64Index([341], dtype='int64')
Int64Index([587], dtype='int64')
Int64Index([404], dtype='int64')
Int64Index([838], dtype='int64')
Int64Index([332], dtype='int64')
Int64Index([

In [11]:
display(muse_data_step)

,RAC1P,SEX,REGION,PINCP,ST,COW,CIT,NATIVITY,MSP,SCHL,DIS,MIL
1,6,2,3,-2615.090605818703,48,None,1,1,5.0,22.0,2,4.0
4,9,2,3,None,12,1.0,4,1,1.0,16.0,2,None
6,1,2,3,None,12,1.0,1,1,6.0,21.0,2,4.0
9,6,1,4,-8500.0,22,None,1,1,6.0,19.0,2,4.0
11,1,1,4,131915.27662908792,36,1.0,1,1,6.0,18.0,2,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,1,2,57638.04670428213,34,1.0,1,1,1.0,1.0,2,4.0
996,1,2,1,-8500.0,50,None,1,1,6.0,1.0,2,1.0
997,1,1,4,-6025.412259414399,39,1.0,1,1,1.0,21.0,2,4.0
998,8,2,3,None,37,1.0,4,1,6.0,16.0,2,4.0


In [12]:
muse_data_step.to_csv('datasets/muse/muse_data_step.csv',index=False)